<Font size =5> Creating a dataset for the past 365 days using a Nepse API

In [ ]:
import requests
import pandas as pd
import datetime
import time

from nepse_scraper import Nepse_scraper

scraper = Nepse_scraper()
end_date = datetime.date.today()
start_date = end_date - datetime.timedelta(days=365)

all_data = []

# loop day by day
current = start_date
while current <= end_date:
    try:
        # fetch one day's snapshot
        daily_response = scraper.get_today_price(current.strftime("%Y-%m-%d"))
        
        # NEW: handle JSON structure with "content" key
        companies = daily_response.get("content", [])
        
        for c in companies:
            all_data.append({
                "date": c.get("businessDate"),
                "symbol": c.get("symbol"),
                "securityId": c.get("securityId"),
                "securityName": c.get("securityName"),
                "open": c.get("openPrice"),
                "high": c.get("highPrice"),
                "low": c.get("lowPrice"),
                "close": c.get("closePrice"),
                "volume": c.get("totalTradedQuantity"),
                "turnover": c.get("totalTradedValue"),
                "prevClose": c.get("previousDayClosePrice"),
                "52wHigh": c.get("fiftyTwoWeekHigh"),
                "52wLow": c.get("fiftyTwoWeekLow"),
                "trades": c.get("totalTrades"),
                "avgPrice": c.get("averageTradedPrice"),
                "marketCap": c.get("marketCapitalization")
            })
        
        print(f"Collected data for {current}")
    
    except Exception as e:
        print(f"Skipped {current}: {e}")
    except KeyboardInterrupt:
        pass
    
    current += datetime.timedelta(days=1)
    time.sleep(0.1)  # be polite to NEPSE

c:\Users\arses\Desktop\Forecast\venv\Lib\site-packages\nepse_scraper\Scraper.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Collected data for 2024-08-24
Collected data for 2024-08-25
Collected data for 2024-08-26
Collected data for 2024-08-27
Collected data for 2024-08-28
Collected data for 2024-08-29
Collected data for 2024-08-30
Collected data for 2024-08-31
Collected data for 2024-09-01
Collected data for 2024-09-02
Collected data for 2024-09-03
Collected data for 2024-09-04
Collected data for 2024-09-05
Collected data for 2024-09-06
Collected data for 2024-09-07
Collected data for 2024-09-08
Collected data for 2024-09-09
Collected data for 2024-09-10
Collected data for 2024-09-12
Collected data for 2024-09-13
Collected data for 2024-09-14
Collected data for 2024-09-15
Collected data for 2024-09-16
Collected data for 2024-09-17
Collected data for 2024-09-18
Collected data for 2024-09-19
Collected data for 2024-09-20
Collected data for 2024-09-21
Collected data for 2024-09-22
Collected data for 2024-09-23
Collected data for 2024-09-24
Collected data for 2024-09-25
Collected data for 2024-09-26
Collected 

: 

: 

<font size = 5> Creating target values for classification task

In [ ]:
import pandas as pd

# Load your dataset
df = pd.read_csv("nepse_full_1year.csv")

# Sort by symbol and date to keep order
df = df.sort_values(by=["symbol", "date"])

# Shift close price by -1 (next day’s close)
df["next_close"] = df.groupby("symbol")["close"].shift(-1)

# Create target column: 1 if next day close is higher, else 0
df["target"] = (df["next_close"] > df["close"]).astype(int)

#featyre engineering: moving average and volatility added as new features/variables
df["ma_5"] = df.groupby("symbol")["close"].transform(lambda x: x.rolling(5).mean())
df["volatility_10"] = df.groupby("symbol")["close"].transform(lambda x: x.pct_change().rolling(10).std())



df.to_csv("nepse_full_1year.csv", index=False)
print(df[["date", "symbol", "close", "next_close", "target"]].head(15))

          date  symbol   close  next_close  target
0   2024-08-25  ACLBSL  1170.1      1170.0       0
1   2024-08-27  ACLBSL  1170.0      1144.9       0
2   2024-08-28  ACLBSL  1144.9      1138.0       0
3   2024-08-29  ACLBSL  1138.0      1081.0       0
4   2024-09-01  ACLBSL  1081.0      1134.2       1
5   2024-09-02  ACLBSL  1134.2      1116.0       0
6   2024-09-03  ACLBSL  1116.0      1095.0       0
7   2024-09-04  ACLBSL  1095.0      1115.0       1
8   2024-09-05  ACLBSL  1115.0      1143.0       1
9   2024-09-08  ACLBSL  1143.0      1104.0       0
10  2024-09-09  ACLBSL  1104.0      1081.0       0
11  2024-09-10  ACLBSL  1081.0      1077.1       0
12  2024-09-11  ACLBSL  1077.1      1089.0       1
13  2024-09-12  ACLBSL  1089.0      1066.0       0
14  2024-09-15  ACLBSL  1066.0      1031.1       0
